# Regression Model A: Field 2

## Install libraries

In [1]:
install.packages('readxl')
install.packages('corrplot')
install.packages('RColorBrewer')
install.packages('car')
install.packages('Metrics')

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Warning message in install.packages("readxl"):
“installation of package ‘readxl’ had non-zero exit status”Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


## Call libraries

In [2]:
library(readxl)
library(corrplot)
library(RColorBrewer)
library(car)
library(Metrics)

corrplot 0.84 loaded
Loading required package: carData


## Import data

In [3]:
d1=read_excel("variable2.xlsx",sheet = 1)
d2=read_excel("variable2.xlsx",sheet = 2)
d3=read_excel("variable2.xlsx",sheet = 3) 
d4=read_excel("variable2.xlsx",sheet = 4)
d5=read_excel("variable2.xlsx",sheet = 5) 

## Field 2 (F2)
Call the data according to the Field. 

In [4]:
dE1=d1[which(d1$`ESTATE CODE`=="E2" ),]
dE2=d2[(d2$Diff ),]
dE3=d3[which(d3$Estate=="E2" ),]
dE4=d4[which(d4$Estate=="E2" ),]
dE5=d5[which(d5$Estate=="E2" ),]

## Variables

Declare the variable and construct new table for easy representation. 

In [5]:
year=dE1$`CALENDER YEAR`
month=dE1$`CALENDER MONTH`
yield=dE1$`CPB TONNAGE`
rainfall=dE1$RAINFALL
foliar=d2$Diff
prun.time=dE3$Frequency
prun.cost=dE3$PruningCost
manuring.time=dE4$Frequency
manuring.cost=dE4$ManuringCost
pd.time=dE5$Frequency
pd.cost=dE5$PDCost

dg1=data.frame(year,month,yield,rainfall,foliar,prun.time,prun.cost,
               manuring.time,manuring.cost,pd.time,pd.cost)

## Correlation

Check the correlations for all variables. 

In [6]:
c1=cor(dg1[,-1:-2])

corrplot(c1, method="number",type="upper",col=brewer.pal(n=8, name="RdYlBu"))
corrplot(c1, type="upper")

## Regression

Fit the data using multiple regression for all variables. 

In [ ]:
a1=lm(yield~rainfall+foliar+prun.time+prun.cost+
        manuring.time+manuring.cost+pd.time+pd.cost,data=dg1)
summary(a1)
plot(a1,which=1)
plot(a1,which=2)


Call:
lm(formula = yield ~ rainfall + foliar + prun.time + prun.cost + 
    manuring.time + manuring.cost + pd.time + pd.cost, data = dg1)

Residuals:
    Min      1Q  Median      3Q     Max 
-100.33  -25.68   -5.78   30.38  116.14 

Coefficients:
                Estimate Std. Error t value Pr(>|t|)    
(Intercept)    1.190e+02  2.931e+01   4.059 0.000263 ***
rainfall       1.883e-02  3.777e-02   0.499 0.621243    
foliar        -3.090e+00  2.200e+00  -1.405 0.168920    
prun.time      7.247e+00  6.330e+00   1.145 0.260011    
prun.cost      4.549e-03  9.727e-03   0.468 0.642881    
manuring.time  1.466e+01  1.225e+01   1.197 0.239319    
manuring.cost -6.018e-04  8.279e-04  -0.727 0.472144    
pd.time       -1.475e+00  1.070e+01  -0.138 0.891158    
pd.cost        1.578e-03  8.466e-03   0.186 0.853176    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 47.94 on 35 degrees of freedom
Multiple R-squared:  0.1931,	Adjusted R-squared:  0.00862

## Significant variable 

Check the significant variables using two ways. 

a.	anova  
Choose the variables that has significant value in which p<0.1.

b.	using stepwise method  
In order to avoid the multicollinearity issues in variables selection, we take into account the variance inflation factors (VIF). We remove the variables that has VIF greater than 4. Then we rebuild the model. 


The stepwise method incorporate with VIF factor is better than anova method in order to     select the significant variables.    

### Using ANOVA

In [ ]:
anova(a1)

### Using stepwise

In [ ]:
selectedMod <- step(a1)
summary(selectedMod)

##Avoid Multicollinearity and Statistical Significance
##Recursively remove variables with variance inflation factors: VIF > 4
# Remove vars with VIF> 4 and re-build model until none of VIFs don't exceed 4.
all_vifs <- car::vif(selectedMod)
print(all_vifs)

while(any(all_vifs > 4)){
  var_with_max_vif <- names(which(all_vifs == max(all_vifs)))  # get the var with max vif
  signif_all <- signif_all[!(signif_all) %in% var_with_max_vif]  # remove
  myForm <- as.formula(paste("ozone_reading ~ ", paste (signif_all, collapse=" + "), sep=""))  # new formula
  selectedMod <- lm(myForm, data=inputData)  # re-build model with new formula
  all_vifs <- car::vif(selectedMod)
}

summary(selectedMod)

## Prediction

Predict with the resample values.

In [ ]:
set.seed(123)
dg3=sample(1:nrow(dg1),12)
dg4=dg1[dg3,]
estimate=predict(selectedMod,newdata=dg4)
estimate
sum.estimate=sum(estimate)
dg5=data.frame(estimate,sum.estimate)